In [75]:
import cv2
import glob
import torch
import torch.cuda
import numpy as np
import os
import pandas as pd
import json
import pickle
import shutil

import matplotlib.pyplot as plt

import Utils.Data.DataGen as DataGen
import Utils.Data.DataLoading as DataLoading
import Utils.PTModel.Models as Models

In [3]:
# Code to generate fixed lode runner (16 x 16 tile instead of 8 x 8) levels using my resizing solution

# lrLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/*.txt"))
# lrLevelStrings = []
# for levelPath in lrLevelPaths:
#     with open(levelPath, "r") as f:
#         levelString = np.array([list(line[:-1]) for line in f.readlines()])
        
#     DataLoading.TextTileToImage(levelString, 16, "../data/tomData/sprites/loderunner", savePath="../data/vglc/Lode Runner/Original_Fixed_Tom", fileNameOverride=levelPath[35:-4], cvtColor=False)

In [76]:
lrLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Original_Fixed_Tom/*.png"))
lrLevels = [cv2.copyMakeBorder(cv2.cvtColor(cv2.imread(levelPath), cv2.COLOR_BGR2RGB), 16, 16, 16, 16, cv2.BORDER_CONSTANT) for levelPath in lrLevelPaths]
lrLevelKernals = [DataGen.ResizeLevel(level, 16, 3, columnWise=False) for level in lrLevels]

Image Height: 384 | Image Width: 544
Image Height % 16: 0 | Image Width % 16: 0
BoxHeigh: 48 | BoxWidth: 48
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 384 | Image Width: 544
Image Height % 16: 0 | Image Width % 16: 0
BoxHeigh: 48 | BoxWidth: 48
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 384 | Image Width: 544
Image Height % 16: 0 | Image Width % 16: 0
BoxHeigh: 48 | BoxWidth: 48
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 384 | Image Width: 544
Image Height % 16: 0 | Image Width % 16: 0
BoxHeigh: 48 | BoxWidth: 48
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 384 | Image Width: 544
Image Height % 16: 0 | Image Width % 16: 0


In [77]:
lrStringLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/*.txt"))

with open("../data/vglc/Lode Runner/Loderunner.json", "r") as f:
    affordanceJson = json.load(f)["tiles"]


with open("../data/tomData/AffordanceTrainDataDictionary.pickle", "rb") as f:
    affordanceDict = pickle.load(f)

levelStrings = []
lrLevelEncodedAffordances = []

for i, levelStringPath in enumerate(lrStringLevelPaths):
    with open(levelStringPath, "r") as f:
        levelString = np.array([list(line[:-1]) for line in f.readlines()])
    levelString = np.pad(levelString, 1, constant_values='.')
    
    lrStringLevelKernals = DataGen.ResizeLevel(levelString, 1, 3, columnWise=False)
    lrStringAffordances = [DataGen.GetAffordances(kernel, affordanceJson, True) for kernel in lrStringLevelKernals]

    lrLevelEncodedAffordances.append([np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in affordanceList]), axis=0) for affordanceList in lrStringAffordances])

lrLevelEncodedAffordances = np.array(lrLevelEncodedAffordances)

Image Height: 24 | Image Width: 34
Image Height % 16: 8 | Image Width % 16: 2
BoxHeigh: 3 | BoxWidth: 3
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 24 | Image Width: 34
Image Height % 16: 8 | Image Width % 16: 2
BoxHeigh: 3 | BoxWidth: 3
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 24 | Image Width: 34
Image Height % 16: 8 | Image Width % 16: 2
BoxHeigh: 3 | BoxWidth: 3
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 24 | Image Width: 34
Image Height % 16: 8 | Image Width % 16: 2
BoxHeigh: 3 | BoxWidth: 3
Start Position: 1
Kernel Size: 3
Kernel Size % 2: 1
Kernel Size % 2 * 2: 2
Height With Kernel: 22 | Width With Kernel: 32
Image Height: 24 | Image Width: 34
Image Height % 16: 8 | Image Width % 16: 2
BoxHeigh: 3 | BoxW

In [78]:
device = "cuda" if torch.cuda.is_available() else "cpu"

@torch.no_grad()
def SaveLevelUnifiedRepresentation(model, levelData, savePath, gameName, affordanceData=None):
    # Add compatability for affordances to be added instead of zeroed arrays for lode runner use
    model.to(device)
    model.eval()

    if not os.path.isdir(savePath):
        os.mkdir(savePath)
    else:
        if not os.path.isdir(f"{savePath}/{gameName}"):
            os.mkdir(f"{savePath}/{gameName}")
        else:
            shutil.rmtree(f"{savePath}/{gameName}")
    

    tiles = []
    embeddings = []

    for i, level in enumerate(levelData):

        imageArray = np.array(level)
        if type(affordanceData) == np.ndarray:
            affordanceArray = affordanceData[i]
        else:
            affordanceArray = np.zeros(shape=(imageArray.shape[0], 13))

        # Adjust 48 x 48 resize to be dynamic for 5 x 5 or larger kernels
        imageArrayTensor = torch.tensor(np.reshape(imageArray, (-1, 3, 48, 48)), dtype=torch.float32).to(device)
        affordanceArrayTensor = torch.tensor(affordanceArray, dtype=torch.float32).to(device)

        embeddingArray = model.encode(imageArrayTensor, affordanceArrayTensor).detach().cpu().numpy()

        centerTiles = imageArray[:, 16:32, 16:32, :]

        np.save(f"{savePath}/{gameName}/level{i}Embedding.npy", embeddingArray)

        tiles.extend(centerTiles)
        embeddings.extend(embeddingArray)

    np.save(f"{savePath}/{gameName}/centerTiles.npy", np.array(tiles))
    np.save(f"{savePath}/{gameName}/embeddings.npy", np.array(embeddings))

In [79]:
model = torch.load("Models/firstTry.pt")

In [81]:
SaveLevelUnifiedRepresentation(model, lrLevelKernals, "testrep", "loderunner", affordanceData=lrLevelEncodedAffordances)